<a href="https://colab.research.google.com/github/leonfrench/polyprotein_stats/blob/yeast_embeddings/notebooks/generate_proportions_yeast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!wget https://github.com/leonfrench/AD-Allen-FDG/raw/master/data/protein_gencode/gencode.v32.pc_translations.fa.gz
#!gzip -d gencode.v32.pc_translations.fa.gz

In [1]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 4.8 MB/s 


In [5]:
#!unzip yeast.all.zip
!ls /yeast.all/| wc -l

ls: cannot access '/yeast.all/': No such file or directory
0


In [6]:
import pandas as pd
import numpy as np
import os
from Bio import SeqIO
from datetime import datetime
import functools
from google.colab import drive
from collections import Counter

In [7]:
directory = "./yeast.all/"
sequence_list = list()
for file in os.listdir(directory):
     filename = os.fsdecode(file)
     if filename.endswith(".fasta"): 
         fasta_sequences = SeqIO.parse(open(os.path.join(directory, filename)),'fasta')
         sequence_list.append(next(fasta_sequences))
         continue
     else:
         continue

In [8]:
len(sequence_list)

5222

In [ ]:
embedding_file_path_processed = './gdrive/MyDrive/data/protein_embeddings_processed/'

In [ ]:
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
#sequence_list = list(SeqIO.parse("gencode.v32.pc_translations.fa", "fasta"))

In [9]:
sequences_strings = [str(x.seq) for x in sequence_list]
sequences_ids = [x.id for x in sequence_list]

In [10]:
sequence_df = pd.DataFrame(  data = {'ids' : sequences_ids, 'sequences': sequences_strings})

In [11]:
#count amino acids
sequence_df = pd.concat([sequence_df.reset_index(drop=True), sequence_df['sequences'].apply(Counter).apply(pd.Series)], axis = 1)

In [13]:
sequence_df

,ids,sequences,M,Q,S,T,V,P,I,A,...,K,R,D,E,L,F,Y,C,H,W
0,YPR030W,MQSTVPIAIASNGNKRDVVQNVSAGDEGDILQRLARNREMISTSLS...,19.0,37.0,149.0,83.0,56.0,69.0,58.0,59.0,...,57.0,65.0,69.0,46.0,87.0,35.0,43.0,11.0,20.0,5.0
1,YPL033C,MKKTIYKVLVSFYQYVGLGKKFHPSHDTVLIIGGSSNELGIELCET...,3.0,6.0,29.0,17.0,13.0,6.0,30.0,6.0,...,27.0,7.0,12.0,22.0,33.0,15.0,10.0,7.0,8.0,1.0
2,YOL148C,MSANSPTGNDPHVFGIPVNATPSNMGSPGSPVNVPPPMNPAVANVN...,18.0,46.0,60.0,42.0,23.0,35.0,22.0,46.0,...,39.0,34.0,18.0,36.0,44.0,17.0,17.0,2.0,11.0,2.0
3,YIR025W,MARALRDISLFNDIRKDQNSAGAKHERYNMRDLRSKKNQHVNGIDD...,10.0,15.0,30.0,9.0,16.0,15.0,18.0,16.0,...,14.0,33.0,39.0,41.0,27.0,10.0,15.0,NaN,9.0,3.0
4,YOR251C,MPLFDLISPKAFVKLVASEKVHRIVPVDATWYLPSWKLDNKVDFLT...,5.0,5.0,26.0,14.0,21.0,26.0,15.0,18.0,...,25.0,10.0,22.0,19.0,25.0,17.0,12.0,4.0,7.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,YGL059W,MSKYQINCIRYRHFLRTSNISQIPDFTKYCIGPVNEELAPYIMETM...,14.0,19.0,37.0,27.0,23.0,22.0,36.0,16.0,...,38.0,20.0,20.0,36.0,57.0,21.0,33.0,5.0,17.0,2.0
5218,YFL038C,MNSEYDYLFKLLLIGNSGVGKSCLLLRFSDDTYTNDYISTIGVDFK...,5.0,9.0,15.0,18.0,14.0,1.0,11.0,8.0,...,18.0,7.0,17.0,12.0,20.0,8.0,9.0,4.0,1.0,2.0
5219,YDL014W,MSFRPGSRGGSRGGSRGGFGGRGGSRGGARGGSRGGFGGRGGSRGG...,8.0,6.0,24.0,6.0,27.0,16.0,18.0,25.0,...,18.0,35.0,12.0,20.0,16.0,10.0,8.0,3.0,7.0,1.0
5220,YDR495C,MVKKKTNNDKGKEVKENEGKLDIDSESSPHERENDKKKTEDDSLRA...,16.0,24.0,82.0,55.0,44.0,26.0,86.0,33.0,...,98.0,30.0,69.0,105.0,123.0,50.0,33.0,16.0,19.0,5.0


In [14]:
#drop the sequence column
#extract gene_symbol again (code reuse from another workbook)
sequence_df['gene_symbol'] = sequence_df['ids']
sequence_df.drop(['ids', 'sequences'], axis=1, inplace=True) 

cols = sequence_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
sequence_df = sequence_df[cols]
#write out

In [15]:
sequence_df['length'] = sequence_df.sum(axis=1, numeric_only=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
sequence_df

,gene_symbol,M,Q,S,T,V,P,I,A,N,...,R,D,E,L,F,Y,C,H,W,length
0,YPR030W,19.0,37.0,149.0,83.0,56.0,69.0,58.0,59.0,111.0,...,65.0,69.0,46.0,87.0,35.0,43.0,11.0,20.0,5.0,1121.0
1,YPL033C,3.0,6.0,29.0,17.0,13.0,6.0,30.0,6.0,14.0,...,7.0,12.0,22.0,33.0,15.0,10.0,7.0,8.0,1.0,281.0
2,YOL148C,18.0,46.0,60.0,42.0,23.0,35.0,22.0,46.0,70.0,...,34.0,18.0,36.0,44.0,17.0,17.0,2.0,11.0,2.0,604.0
3,YIR025W,10.0,15.0,30.0,9.0,16.0,15.0,18.0,16.0,28.0,...,33.0,39.0,41.0,27.0,10.0,15.0,NaN,9.0,3.0,368.0
4,YOR251C,5.0,5.0,26.0,14.0,21.0,26.0,15.0,18.0,10.0,...,10.0,22.0,19.0,25.0,17.0,12.0,4.0,7.0,6.0,304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,YGL059W,14.0,19.0,37.0,27.0,23.0,22.0,36.0,16.0,28.0,...,20.0,20.0,36.0,57.0,21.0,33.0,5.0,17.0,2.0,491.0
5218,YFL038C,5.0,9.0,15.0,18.0,14.0,1.0,11.0,8.0,12.0,...,7.0,17.0,12.0,20.0,8.0,9.0,4.0,1.0,2.0,206.0
5219,YDL014W,8.0,6.0,24.0,6.0,27.0,16.0,18.0,25.0,5.0,...,35.0,12.0,20.0,16.0,10.0,8.0,3.0,7.0,1.0,327.0
5220,YDR495C,16.0,24.0,82.0,55.0,44.0,26.0,86.0,33.0,69.0,...,30.0,69.0,105.0,123.0,50.0,33.0,16.0,19.0,5.0,1011.0


In [18]:
#drop x and u
#sequence_df.drop(['X', 'U'], axis=1, inplace=True) 
#fill na
sequence_df.fillna(0, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5182: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
#change to proportion

In [19]:
cols = set(sequence_df.columns.values).difference(('gene_symbol','length'))

In [20]:
sequence_df[[col for col in cols]] = sequence_df[cols].div(sequence_df['length'], axis='index')

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [21]:
sequence_df

,gene_symbol,M,Q,S,T,V,P,I,A,N,...,R,D,E,L,F,Y,C,H,W,length
0,YPR030W,0.016949,0.033006,0.132917,0.074041,0.049955,0.061552,0.051740,0.052632,0.099019,...,0.057984,0.061552,0.041035,0.077609,0.031222,0.038359,0.009813,0.017841,0.004460,1121.0
1,YPL033C,0.010676,0.021352,0.103203,0.060498,0.046263,0.021352,0.106762,0.021352,0.049822,...,0.024911,0.042705,0.078292,0.117438,0.053381,0.035587,0.024911,0.028470,0.003559,281.0
2,YOL148C,0.029801,0.076159,0.099338,0.069536,0.038079,0.057947,0.036424,0.076159,0.115894,...,0.056291,0.029801,0.059603,0.072848,0.028146,0.028146,0.003311,0.018212,0.003311,604.0
3,YIR025W,0.027174,0.040761,0.081522,0.024457,0.043478,0.040761,0.048913,0.043478,0.076087,...,0.089674,0.105978,0.111413,0.073370,0.027174,0.040761,0.000000,0.024457,0.008152,368.0
4,YOR251C,0.016447,0.016447,0.085526,0.046053,0.069079,0.085526,0.049342,0.059211,0.032895,...,0.032895,0.072368,0.062500,0.082237,0.055921,0.039474,0.013158,0.023026,0.019737,304.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,YGL059W,0.028513,0.038697,0.075356,0.054990,0.046843,0.044807,0.073320,0.032587,0.057026,...,0.040733,0.040733,0.073320,0.116090,0.042770,0.067210,0.010183,0.034623,0.004073,491.0
5218,YFL038C,0.024272,0.043689,0.072816,0.087379,0.067961,0.004854,0.053398,0.038835,0.058252,...,0.033981,0.082524,0.058252,0.097087,0.038835,0.043689,0.019417,0.004854,0.009709,206.0
5219,YDL014W,0.024465,0.018349,0.073394,0.018349,0.082569,0.048930,0.055046,0.076453,0.015291,...,0.107034,0.036697,0.061162,0.048930,0.030581,0.024465,0.009174,0.021407,0.003058,327.0
5220,YDR495C,0.015826,0.023739,0.081108,0.054402,0.043521,0.025717,0.085064,0.032641,0.068249,...,0.029674,0.068249,0.103858,0.121662,0.049456,0.032641,0.015826,0.018793,0.004946,1011.0


In [22]:
#combine multiple genes into single rows
sequence_df = sequence_df.groupby('gene_symbol').agg('mean')
sequence_df = sequence_df.reset_index()

In [23]:
sequence_df.shape

(5222, 22)

In [24]:
sequence_df

,gene_symbol,M,Q,S,T,V,P,I,A,N,...,R,D,E,L,F,Y,C,H,W,length
0,Q0045,0.039326,0.016854,0.080524,0.058052,0.071161,0.046816,0.091760,0.082397,0.050562,...,0.016854,0.024345,0.016854,0.127341,0.082397,0.044944,0.001873,0.029963,0.018727,534.0
1,Q0050,0.063549,0.016787,0.053957,0.073141,0.049161,0.035971,0.077938,0.037170,0.082734,...,0.055156,0.040767,0.034772,0.089928,0.035971,0.046763,0.015588,0.020384,0.009592,834.0
2,Q0055,0.042155,0.015222,0.073770,0.046838,0.058548,0.029274,0.076112,0.035129,0.086651,...,0.052693,0.038642,0.036300,0.103044,0.044496,0.046838,0.024590,0.035129,0.007026,854.0
3,Q0060,0.045783,0.021687,0.069880,0.057831,0.053012,0.016867,0.060241,0.048193,0.127711,...,0.033735,0.031325,0.036145,0.113253,0.038554,0.074699,0.007229,0.014458,0.014458,415.0
4,Q0065,0.053957,0.028777,0.071942,0.059353,0.053957,0.025180,0.077338,0.052158,0.082734,...,0.023381,0.035971,0.023381,0.120504,0.064748,0.041367,0.005396,0.014388,0.021583,556.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217,YPR194C,0.027366,0.025086,0.064994,0.052452,0.060433,0.049031,0.090080,0.060433,0.054732,...,0.021665,0.046750,0.031927,0.080958,0.074116,0.075257,0.014823,0.019384,0.031927,877.0
5218,YPR198W,0.025783,0.027624,0.099448,0.057090,0.051565,0.025783,0.103131,0.073665,0.042357,...,0.012891,0.022099,0.029466,0.125230,0.084715,0.033149,0.020258,0.016575,0.016575,543.0
5219,YPR199C,0.023810,0.030612,0.088435,0.064626,0.044218,0.047619,0.054422,0.051020,0.068027,...,0.047619,0.044218,0.071429,0.102041,0.064626,0.020408,0.027211,0.020408,0.003401,294.0
5220,YPR200C,0.007692,0.061538,0.084615,0.069231,0.076923,0.015385,0.069231,0.046154,0.038462,...,0.053846,0.046154,0.069231,0.076923,0.069231,0.015385,0.023077,0.030769,0.015385,130.0


In [25]:
sequence_df = sequence_df.reset_index(drop=True)

In [27]:
sequence_df.to_csv("protein_ID_summarized_proportions.csv", index=False)

In [26]:
embedding_file_path_processed

NameError: ignored